# CNN VGG16

### Import libraries to be used





In [27]:
import numpy as np
import pandas as pd
from keras import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation,Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from statistics import mean

### Import dataset

In [28]:
train_data = np.genfromtxt('/content/drive/MyDrive/BigData/dataset/sign_mnist_train.csv', delimiter=',')
test_data = np.genfromtxt('/content/drive/MyDrive/BigData/dataset/sign_mnist_test.csv', delimiter=',')

In [29]:
X_train = train_data[1:, 1:]
y_train = train_data[1:, 0]

X_test = test_data[1:, 1:]
y_test = test_data[1:, 0]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(27455, 784) (27455,)
(7172, 784) (7172,)


### Data preprocessing

In [30]:
# normalise pixel values from 0-255 to 0-1
X_train = X_train / 255.0
X_test = X_test / 255.0

# reshape 1d array of 784 to 2d array of size 28x28
X_train = X_train.reshape(len(X_train), 28, 28)
X_test = X_test.reshape((len(X_test)), 28, 28)

# expand 28x28 to 32x32
X_train = tf.constant(X_train)
X_train = X_train[..., tf.newaxis]

X_test = tf.constant(X_test)
X_test = X_test[..., tf.newaxis]

X_train = tf.image.resize(X_train, [32, 32]).numpy()
X_test = tf.image.resize(X_test, [32, 32]).numpy()

# increase channels from 1 to 3 for better suitability to model
X_train = X_train[:, :,:, 0]
X_test = X_test[:, :,:, 0]

X_train = np.repeat(X_train[..., np.newaxis], 3, -1)
X_test = np.repeat(X_test[..., np.newaxis], 3, -1)

y_train = np.repeat(y_train[..., np.newaxis], 1, -1)
y_test = np.repeat(y_test[..., np.newaxis], 1, -1)

print(X_train.shape)  # (64, 224, 224, 3)
print(X_test.shape)  # (64, 224, 224, 3)
print(y_train.shape)
print(y_test.shape)


(27455, 32, 32, 3)
(7172, 32, 32, 3)
(27455, 1)
(7172, 1)


### Initialize the model

In [31]:
# Initialize the base model
base_model_vgg16 = VGG16(include_top = False, weights= 'imagenet', input_shape = (32,32,3), classes = y_train.shape[1])

In [32]:
# Adding final layers to model
model = Sequential()
model.add(base_model_vgg16) 
model.add(Flatten())
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dense(128,activation=('relu')))
model.add(Dense(25,activation=('softmax')))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 1024)              525312    
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
 dense_12 (Dense)            (None, 256)               131328    
                                                                 
 dense_13 (Dense)            (None, 128)               32896     
                                                                 
 dense_14 (Dense)            (None, 25)               

### Initialize K-Fold

In [33]:
kf = StratifiedKFold(n_splits=5)

### Compile the model

In [34]:
model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [35]:
def to_binary_categorical(array):
    new_array = []

    for i in range(len(array)):
        content = [0] * 25
        index = np.argmax(array[i])
        content[index] = 1
        new_array.append(content)

    return new_array

### K-Fold Cross Validation

In [36]:
f1_scores = []
fold_no = 1

for train_index, test_index in kf.split(X_train, y_train):
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # one-hot
    y_train_fold = to_categorical(y_train_fold)
    y_test_fold = to_categorical(y_test_fold)

    # fit and predict
    model.fit(x=X_train_fold, y=y_train_fold)
    y_predicted = model.predict(x=X_test_fold)

    # convert predicted float values to binary values
    y_predicted = to_binary_categorical(y_predicted)
    
    # calculate score
    score = f1_score(y_test_fold, y_predicted, average='micro')
    f1_scores.append(score)
    print('Score for fold no.', fold_no ,': ',score)
    fold_no+=1

print(f1_scores)
print('Average F1 Score:', mean(f1_scores))

687/687 [==============================] - 20s 28ms/step - loss: 0.8819 - accuracy: 0.7297
Score for fold no. 1 :  0.9989073028592241
687/687 [==============================] - 19s 28ms/step - loss: 0.0175 - accuracy: 0.9963
Score for fold no. 2 :  1.0
687/687 [==============================] - 19s 27ms/step - loss: 0.4862 - accuracy: 0.8694
Score for fold no. 3 :  0.9994536514296121
687/687 [==============================] - 19s 28ms/step - loss: 0.0040 - accuracy: 0.9989
Score for fold no. 4 :  0.9998178838098707
687/687 [==============================] - 19s 28ms/step - loss: 0.1818 - accuracy: 0.9572
Score for fold no. 5 :  0.9956292114368968
[0.9989073028592241, 1.0, 0.9994536514296121, 0.9998178838098707, 0.9956292114368968]
Average F1 Score: 0.9987616099071207


### Final result

In [37]:
y_test = to_categorical(y_test)

y_predicted = model.predict(x=X_test)
y_predicted = to_binary_categorical(y_predicted)

score = f1_score(y_test, y_predicted, average='micro')
print('F1 Score:', score)

F1 Score: 0.9637479085331846
